
***

# Tutoral outline


_1-overview.ipynb_

**1. Introductory concepts**

    1.1 When we say 'geothermal' we mean...

    1.2 Introduction to geothermal wells

    1.3 Introduction to completion tests


**2. Import and munge test data**
    
    2.1 Pump (flowrate) data
        
    2.1.1 Import flowrate data using Pandas

    2.1.2 Rename columns

    2.1.3 Convert datetime string into Python datetime 

    2.1.4 Add a flowrate field in our desired units

    2.2 Pressure, temperature, spinner tool data

    2.1.1 Import pts data using Pandas

    2.2.2 Rename selected column headers

    2.2.3 Drop units row

    2.2.4 Recognise bad datetime data & how to handle excel datetime floats

    2.2.5 Drop unwanted columns from dataframe

    2.2.6 Correct the data types

**3. Overview the completion test data**


_2-temprature.ipynb_

**4. Introductory concepts**

    4.1 The temperature when?

    4.2 Uses of temperature logs in geothermal

**5. Extract and evaluate temperature from completion test data**

    5.1 Use functions to import and mudge data

    5.2 Extract the desired temperature log

    5.2.1 Use ipywidget sliders to interactively select desired data

    5.3 Use XX to generate a new dataframe of the desired data

    5.4 Evaulate the temprature log

_3-injectivity.ipynb_

_2-feedzones.ipynb_



***

# 1. Introduction

XXX

## 1.1 When we say 'geothermal' we mean...

conventional geothermal resources where fluid is rising buoyantly through the crust to form a reservoir that can be intersected by drilling. Where there is sufficient temperature to discharge with the energy of boil or hydrostatic head, wells will self-discharge. Otherwise, wells are pumped. The liquid reservoir is typically hot hydrostatic (i.e., at a lower pressure than the regional cold hydrostatic gradient). We do not discuss engineered geothermal systems, which are also referred to as 'hot dry rock', or the low-temperature sedimentary aquifer systems. However, the testing methods used in those settings are similar to what is described in this tutorial. 

### Include the temperature range table with a note for self-discharging vs pumped

## 1.2 Introduction to geothermal wells

Long open-hole lengths with perforated liners (with a photo of the liner)

Flow because of boiling (rather than from pressure)

Drilled, completion test, heated, production test 

Photo of the tool

## 1.3 Introduction to completion tests

A completion test is ... 



**Katie to finish text?**


***

# 2. Import and munge test data

We are working with two sources of data:

1. Pumps operating at the surface (datetime and flow rate)
2. PTS tool in the well (datetime, pressure, temperature, spinner, other tool information)

The following steps take the raw data that has been provided as excel files, imports them as a Pandas dataframe (one called flowrate and the other called pts), and then do some manipulation to generate a nice clean dataset that we can work with.

We will see that data format and data quality are common challenges, especially when datetime is involved. 

In [ ]:
import pandas as pd

***

## 2.1. Pump (flowrate) data

We will turn the excel data into a Pandas dataframe that includes:
- A datetime field we can use with the tool data
- Flowrate in the units we need

### 2.1.1 Import flowrate data using Pandas

We read the data in with the header location defined as row 1. This automatically skips row 0 which just contains some text that we don’t want.

In [ ]:
flowrate = pd.read_excel(r'Data-FlowRate.xlsx', header=1) 

### Some useful Pandas methods

We will regularly use a set of Pandas methods to check our data as we munge it


Display the first few rows of the dataframe
> df.head() _or_ df.head(n) _to show n rows_

Display the last few rows of the dataframe
> df.tail() _or_ df.tail(n) _to show n rows_

Useful information like the dataframe shape and the data types
> df.info() 

A list of the column headers
> df.columns 

Data types in each coloumn
> df.dtypes

Shape of the dataframe
> df.shape

### 2.1.2 Rename coloums

We do this to remove spaces from the column headers. 

This step is not compulsory, but the presence of spaces in the column names dictates the code format. Once a column has been created, we can interchangeably use the flowrate.raw_datetime or flowrate\['raw_datetime'\] notation. If there are spaces in the column names, the df.column_name notation will not work and we must use the df\['column_name'\] format.


In [ ]:
flowrate.columns = ['raw_datetime','flow_Lpm'] 

flowrate.head(2)

### 2.1.3 Convert datetime string into Python datetime 

The datetime is in [ISO format](https://en.wikipedia.org/wiki/ISO_8601) and, because it includes the timezone, this is the most globally useful datetime format. 

> 2020-12-11T06:00:00+13:00 

where T indicates the time data and +13:00 is the time shift between UTC and New Zealand where the pumps were recording data. 

In [ ]:
flowrate.info()

Look at the output above and note that the _raw_datetime_ data type is _object_ and not _datetime_

We will use a two-step process to parse this date:

1. Convert it to a datetime object with the default UTC timezone
2. Convert the datetime object to local time in New Zealand so it can be combined with the pts tool data

In [ ]:
# use a method that recognises the ISO format to parse the datetime
from datetime import datetime

list = []
for date in flowrate['raw_datetime']:
    newdate = datetime.fromisoformat(date)
    list.append(newdate)
flowrate['ISO_datetime'] = list

flowrate.tail(2)

The method fromisoformate() is new in version Python 3.7

More information can be found [here](https://docs.python.org/3/library/datetime.html) and [here](https://pythontic.com/datetime/date/fromisoformat)

In [ ]:
flowrate.info()

Now we can see that the data type for the ISO_datetime column is a UTC datetime: datetime64\[ns, UTC+13:00\]

But we need it in local NZ time to combine with the PTS tool data. Luckily, the datetime.strftime method recognises the timezone in the datetime object and does the conversion for us. 

In [ ]:
list = []
for date in flowrate.ISO_datetime:
    newdate = pd.to_datetime(datetime.strftime(date,'%Y-%m-%d %H:%M:%S'))
    list.append(newdate)
flowrate['datetime'] = list

flowrate.head()

In [ ]:
flowrate.info()

Now we have a Python datetime object that we can combine with the PTS data which does not have timezone information.  

### 2.1.4 Add a flowrate field in our desired units

Conversions for typical geothermal flowrate units:

- L/sec to t/hr 3.6
- L/min to t/hr 0.060

In [ ]:
flowrate['flow_tph'] = flowrate.flow_Lpm * 0.060

flowrate.head(2) # 23

### 2.1.5 Trim excess coloumns from the dataframe

Now we just drop the columns that we don't want from the dataframe

In [ ]:
flowrate.drop(columns = ['raw_datetime', 'flow_Lpm', 'ISO_datetime'], inplace = True)

flowrate.head(2)

### inplace = True

Each time we do something something like drop columns, we generate a new Pandas dataframe. The argument inplace=True overwrites the old dataframe with the new one. We could have achieved the same thing with assignment:

> flowrate = flowrate.drop(columns = \['raw_datetime', 'flow_Lpm', 'ISO_datetime'\])

***
## 2.2 Pressure, temperature, spinner tool data

We will use a similar process as above to import and munge the pressure, temperature, and spinner (PTS) tool data. However, the PTS dataset includes an addional challenge generated by bad data.


### 2.1.1 Import pts data using Pandas

In [ ]:
pts = pd.read_excel(r'Data-PTS.xlsx') 

If you run the cell below, you will see that we have a lot of data in the pts dataframe (101294 rows and 26 columns). 

Efficient handling of large datasets is one of the reasons why Python is better for these kinds of analysis than excel. 

In [ ]:
pts.shape

In [ ]:
pts.head(3)

### 2.2.2 Rename selected coloumn headders

The dataframe has a row of units below the header row that we need to remove. But because we want to know the units of our data, we will rename key columns to remove spaces and include units before dropping the row containing the units.

In [ ]:
# dictionary method for renaming some coloums
dict = {
    'DEPTH':'depth_m',
    'SPEED': 'speed_mps',
    'Cable Weight': 'cweight_kg',
    'WHP': 'whp_barg',
    'Temperature': 'temp_degC',
    'Pressure': 'pressure_bara',
    'Frequency': 'frequency_hz'
}

pts.rename(columns=dict, inplace=True)

pts.head(3)

### 2.2.3 Drop units row

Note that we are using the index value to drop the row of units and then we remake the index. Therefore, if we run this cell over and over, it will keep dropping the first row of data. 

If you are unsure what state your dataframe is in or how many times you have run the cell below, it is useful to run the process from the start again (i.e., go back to step _2.1.1 Import pts data using Pandas_ and work down from there). 

In [ ]:
# drop units row using the index value of that row
pts.drop(0, inplace=True)

# re-index the dataframe
pts.reset_index(drop=True, inplace=True)

# WARNING if you run this cell again, it will drop the first line of data


In [ ]:
pts.head(2)

### 2.2.4 Recognise bad datetime data & how to handle excel datetime floats

If you investigate the _Time_ column in our pts dataframe, you'll find that the hours are all set to 00 despite the test running for nearly 12 hours and starting at around 9 in the morning. Unfortunately, these kinds of issues with bad data are common and they are why it's important to QA/QC of data when it is acquired. 

We will (reluctantly) use the excel timestamp to generate a new datetime.

An excel timestamp is a floating-point number that reflects the number of days (or part thereof) since a specified start day. There are two systems that start counting from different dates (1900 and 1904) and some fundamental bugs in the way they are implemented. The start date is stored in the excel workbook and we need to know which system is used to correctly generate a datetime. Check out [this documentation](https://openpyxl.readthedocs.io/en/stable/datetime.html) for more information on the challenges of excel timestamps. **The short version is that we should avoid using excel float time whenever possible.** Unfortunately, it is not possible here because of the bad data.

Because we have an xlsx file, will use the openpyxl package to determine the start date in the workbook and parse the excel float to datetime. If you have an xls file, then the xlrd module may be used to achieve the same result.

In [ ]:
# first check the start day system in the workbook
import openpyxl

wb = openpyxl.load_workbook(filename="Data-PTS.xlsx")
if wb.epoch == openpyxl.utils.datetime.CALENDAR_WINDOWS_1900: 
    print("This workbook is using the 1900 date system.")
if wb.epoch == openpyxl.utils.datetime.CALENDAR_MAC_1904: 
    print("This workbook is using the 1904 date system.")


As we have the 1900 date system, we can use the default arguments in the from_excel method in the openpyxl utilities. Refer to [the docs](https://openpyxl.readthedocs.io/en/stable/api/openpyxl.utils.datetime.html) for more information.

In [ ]:
list = []
for date in pts.Timestamp:
    newdate = openpyxl.utils.datetime.from_excel(date)
    list.append(newdate)
pts['datetime'] = list

In [ ]:
pts.head(3)

### 2.2.5 Drop unwanted columns from dataframe

There are a lot of columns we do not need for our analysis, so we will drop these


In [ ]:
pts.columns

In [ ]:
pts.drop(columns = ['Date', 'Time', 'Timestamp','Reed 0',
       'Reed 1', 'Reed 2', 'Reed 3', 'Battery Voltage', 
       'PRT Ref Voltage','SGS Voltage', 'Internal Temp 1', 
       'Internal Temp 2', 'Internal Temp 3','Cal Temp', 
       'Error Code 1', 'Error Code 2', 'Error Code 3',
       'Records Saved', 'Bad Pages',], inplace = True)

In [ ]:
pts.head(3)

### 2.2.6 Correct the data types

Run the cell below and note that the data types are 'object'. This is because we had a mix of text and numeric when importing these data (remember the units row). We convert the dtype to numeric using the pd.to_numeric method.

In [ ]:
pts.dtypes

In [ ]:
pts[
    ['depth_m', 'speed_mps','cweight_kg','whp_barg','temp_degC','pressure_bara','frequency_hz']
] = pts[
    ['depth_m','speed_mps','cweight_kg','whp_barg','temp_degC','pressure_bara','frequency_hz']
    ].apply(pd.to_numeric)

pts.dtypes

***

## We are finished importing and munging the data!

We now have flowrate and pts dataframes that we can work with for the rest of our analysis. 

The steps taken above to import and munge the data are wrapped into custom functions in uitlities.py. 

In the rest of this tutorial, we will call these functions rather than stepping through the method again. 

> from uitlities import*

> flowrate = read_flowrate(r'PTS-2-injection-rate.xlsx')

> pts = read_pts(r'PTS-2.xlsx')


***

# 3. Overview of the completion test data

Katie is making a richer plot for here


In [ ]:
import matplotlib.pyplot as plt 
import matplotlib.dates as mdates

In [ ]:
fig, (ax1, ax2, ax3, ax4, ax5, ax6) = plt.subplots(6, 1,figsize=(10,15),sharex=True)
ax1.set_title('Entire completion test dataset',y=1.1,fontsize=15)

ax1.plot(flowrate.datetime, flowrate.flow_tph, label='Surface pump flowrate', 
    c='k', linewidth=0.8, marker='.')
ax1.set_ylabel('Surface flowrate [t/hr]')
ax1.set_ylim(0,150)
 
ax2.plot(pts.datetime, pts.depth_m, label='PTS tool depth', 
    c='k', linewidth=0.8)
ax2.set_ylabel('PTS tool depth [m]')
ax2.set_ylim(1000,0)
 
ax3.plot(pts.datetime, pts.pressure_bara, label='PTS pressure', 
    c='tab:blue', linewidth=0.8)
ax3.set_ylabel('PTS pressure [bara]')
 
ax4.plot(pts.datetime, pts.temp_degC, label='PTS temperature', 
    c='tab:red', linewidth=0.8)
ax4.set_ylabel('PTS temperature')
 
ax5.plot(pts.datetime, pts.frequency_hz, label='PTS impeller frequency', 
    c='tab:green', linewidth=0.8)
ax5.set_ylim(-30,30)
ax5.set_ylabel('PTS impeller frequency [hz]')
# 1 hz = 60 rpm

ax6.plot(pts.datetime, pts.speed_mps, label='PTS tool speed', 
    c='tab:orange', linewidth=0.8)
ax6.set_ylim(-2,2)
ax6.set_ylabel('PTS tool speed [mps]')
 
ax6.set_xlabel('Time [hh:mm]')
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
 
for ax in [ax1,ax2,ax3,ax4,ax5,ax6]:
    ax.grid()
 
plt.show();

***

The next notebook in this tutoral is 2-temperature.ipynb

***

<p><center>© 2020 <a href="https://www.cubicearth.nz/">Irene Wallis</a> and <a href="https://www.linkedin.com/in/katie-mclean-25994315/">Katie McLean</a> <a href="https://creativecommons.org/licenses/by/4.0/">CC-BY</a></center></p>
